In [1]:
import threading
import json
import time
from kafka import KafkaProducer, KafkaConsumer

In [2]:
class Counter:
    def __init__(self):
        self._lock = threading.Lock()
        self.reset()
        
    def reset(self):
        with self._lock:
            self.counter = 0
            
    def incr(self, value):
        with self._lock:
            self.counter += value
            
    def get(self):
        return self.counter

In [3]:
# Connect to kafka broker running in your local host (docker). Change this to your kafka broker if needed
kafka_broker = 'localhost:9092'
data_topic = 'data'
notification_topic = 'notification'

In [4]:
producer = KafkaProducer(bootstrap_servers=[kafka_broker])

In [5]:
consumer = KafkaConsumer(
     bootstrap_servers=[kafka_broker],
     enable_auto_commit=True,
     value_deserializer=lambda x: x.decode('utf-8'))
consumer.subscribe(data_topic)

In [6]:
def monitor_thread(interval, counters, topic):
    print('[monitor] starting')
    while True:
        time.sleep(interval)
        for id in counters:
            count = counters[id].get()
            counters[id].reset()
            s = '{} - {} messages during last {} seconds'.format(id, count, interval)
            producer.send(topic, s.encode('utf-8'))
        print('send notificiation', flush=True)

In [ ]:
monitor_interval = 10
counters = {}
monitor = threading.Thread(target=monitor_thread, args=(monitor_interval, counters, notification_topic), daemon=True)
monitor.start()

[monitor] starting


send notificiation
send notificiation
send notificiation
send notificiation
send notificiation
send notificiation
send notificiation
send notificiation
send notificiation
send notificiation
send notificiation
send notificiation
send notificiation
send notificiation
send notificiation
send notificiation
send notificiation
send notificiation
send notificiation
send notificiation
send notificiation
send notificiation
send notificiation
send notificiation
send notificiation
send notificiation
send notificiation
send notificiation
send notificiation
send notificiation
send notificiation
send notificiation
send notificiation
send notificiation
send notificiation
send notificiation


In [11]:
for message in consumer:
    m = message.value
    data = json.loads(m)
    if 'id' in data:
        print(data, flush=True)
        if data['id'] not in counters:
            counters[data['id']] = Counter()
        counters[data['id']].incr(1)
    else:
        print(data, flush=True)

{'id': 'sensor-B', 'timestamp': 1742892516, 'value': 29}
{'id': 'sensor-B', 'timestamp': 1742892518, 'value': 32}
{'id': 'sensor-A', 'timestamp': 1742892518, 'value': 15}
{'id': 'sensor-B', 'timestamp': 1742892520, 'value': 31}
{'id': 'sensor-A', 'timestamp': 1742892521, 'value': 34}
{'id': 'sensor-B', 'timestamp': 1742892522, 'value': 30}
{'id': 'sensor-B', 'timestamp': 1742892524, 'value': 30}
{'id': 'sensor-A', 'timestamp': 1742892524, 'value': 24}
{'id': 'sensor-B', 'timestamp': 1742892526, 'value': 28}
{'id': 'sensor-A', 'timestamp': 1742892527, 'value': 21}
{'id': 'sensor-B', 'timestamp': 1742892528, 'value': 29}
{'id': 'sensor-B', 'timestamp': 1742892530, 'value': 29}
{'id': 'sensor-A', 'timestamp': 1742892531, 'value': 23}
{'id': 'sensor-B', 'timestamp': 1742892532, 'value': 31}
{'id': 'sensor-A', 'timestamp': 1742892533, 'value': 27}
{'id': 'sensor-B', 'timestamp': 1742892535, 'value': 31}
{'id': 'sensor-A', 'timestamp': 1742892537, 'value': 16}
{'id': 'sensor-B', 'timestamp':

KeyboardInterrupt: 